In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import torch.optim
import pandas as pd

In [3]:
main_path = "/content/gdrive/My Drive/Uni/NLP/"
data_folder = os.path.join(main_path,"data/")
train_data_name = "train.csv"
data_path = os.path.join(data_folder, train_data_name)
print(data_path)

/content/gdrive/My Drive/Uni/NLP/data/train.csv


In [4]:
datF = pd.read_csv(data_path)

#Take only relevant columns
datRelevant = datF[['user_review', 'user_suggestion']]

train_texts, test_texts, train_labels, test_labels = train_test_split(datRelevant['user_review'], datRelevant['user_suggestion'], test_size=0.25)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts, labels):
  tokenized_batch = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")
  labels = torch.tensor(labels.tolist())
  return tokenized_batch, labels

class ReviewDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.encodings.items()}
    item['labels'] = self.labels[idx]
    return item

  def __len__(self):
    return len(self.labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
train_encodings, train_labels = tokenize_data(train_texts, train_labels)
test_encodings, test_labels = tokenize_data(test_texts, test_labels)
train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)
def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(data_loader):
        optimizer.zero_grad()

        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 10 == 0:
            print(f"Batch {batch_idx + 1}/{len(data_loader)}, Loss: {loss.item():.4f}")

    return total_loss / len(data_loader)

def eval_model(model, data_loader):
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for batch in data_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == batch['labels']).sum().item()
        total += len(batch['labels'])
  return correct / total

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

cuda


In [10]:
for epoch in range(6):  #Training for 6 epoch
    train_loss = train_epoch(model, train_loader, optimizer)
    test_accuracy = eval_model(model, test_loader)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.2f}, Validation Accuracy: {test_accuracy:.2f}")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch 10/820, Loss: 0.5842
Batch 20/820, Loss: 0.5948
Batch 30/820, Loss: 0.4933
Batch 40/820, Loss: 0.3991
Batch 50/820, Loss: 0.6822
Batch 60/820, Loss: 0.3573
Batch 70/820, Loss: 0.7367
Batch 80/820, Loss: 0.3047
Batch 90/820, Loss: 0.4583
Batch 100/820, Loss: 0.3712
Batch 110/820, Loss: 0.6551
Batch 120/820, Loss: 0.4147
Batch 130/820, Loss: 0.4196
Batch 140/820, Loss: 0.2816
Batch 150/820, Loss: 0.4537
Batch 160/820, Loss: 0.3662
Batch 170/820, Loss: 0.2725
Batch 180/820, Loss: 0.5031
Batch 190/820, Loss: 0.5358
Batch 200/820, Loss: 0.2439
Batch 210/820, Loss: 0.5274
Batch 220/820, Loss: 0.5215
Batch 230/820, Loss: 0.4969
Batch 240/820, Loss: 0.2821
Batch 250/820, Loss: 0.3264
Batch 260/820, Loss: 0.0721
Batch 270/820, Loss: 0.2211
Batch 280/820, Loss: 0.2734
Batch 290/820, Loss: 0.3750
Batch 300/820, Loss: 0.2318
Batch 310/820, Loss: 0.2425
Batch 320/820, Loss: 0.2256
Batch 330/820, Loss: 0.1626
Batch 340/820, Loss: 0.4202
Batch 350/820, Loss: 0.2455
Batch 360/820, Loss: 0.2420
B

In [11]:
model_path = os.path.join(main_path, "models/")

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/gdrive/My Drive/Uni/NLP/models/tokenizer_config.json',
 '/content/gdrive/My Drive/Uni/NLP/models/special_tokens_map.json',
 '/content/gdrive/My Drive/Uni/NLP/models/vocab.txt',
 '/content/gdrive/My Drive/Uni/NLP/models/added_tokens.json')

In [12]:
import torch.nn as nn

def eval_model_two(model, data_loader, device):
    model.eval()
    total = 0
    correct = 0
    losses = []
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            loss = nn.functional.cross_entropy(outputs.logits, batch['labels'], reduction='mean')
            losses.append(loss.item())
            correct += (predictions == batch['labels']).sum().item()
            total += len(batch['labels'])
    avg_loss = sum(losses) / len(losses)
    print(f"Validation Loss: {avg_loss:.4f}")
    return correct / total

test_accuracy_two = eval_model_two(model, test_loader, 'cuda')
print(test_accuracy_two)

Validation Loss: 0.4934
0.8902606310013718


In [ ]:
label_counts = datF['user_suggestion'].value_counts()
print(label_counts)

user_suggestion
1    9968
0    7526
Name: count, dtype: int64
